In [1]:
import os
import torch
import time
import json
import csv
import shutil
import pandas as pd
import argparse
import jieba
jieba.load_userdict('dic.txt')
from openai import OpenAI
from utils import (read_docx_files, process_docx_files_2_para, process_docx_files_2_sents, process_docx_files_sents_2_longer, process_docx_files_year,
                   extract_context, clean_json_delimiters, process_docx_files_2_sents_WithNum, tender_document, tender_document_2_para, split_sentences)
from check_duplicate_in_sent import detect_duplicate_phrases, longest_string

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.386 seconds.
Prefix dict has been built successfully.


In [ ]:
from sentence_transformers.util import cos_sim  
from sentence_transformers import SentenceTransformer as SBert
model = SBert('paraphrase-multilingual-MiniLM-L12-v2')

In [3]:
type1_1 = pd.read_excel('../answer_final-1-副本/answers_type1-常识错误-不未错误-备份.xlsx')
type1_1 = type1_1.drop_duplicates(subset=['id', 'sent_id'], keep='last').reset_index(drop=True)
white_list_in = ['用人单位', '信用中国', '未划分标包']
type1_1 = type1_1[~type1_1['sent'].str.contains('|'.join(white_list_in))]
type1_1 = type1_1[~type1_1['id'].str.startswith('新能源')].reset_index(drop=True)

In [8]:
type1_2 = pd.read_excel('../answer_final-1-副本/answers_type1-常识错误-时间错误-备份.xlsx')
type1_2 = type1_2[type1_2['possible_error_sent'] != '时间：5个工作日']
type1_2 = type1_2[~(type1_2['possible_error_sent'].str.contains('投标文件') & type1_2['sent'].str.contains('2024'))]
type1_2 = type1_2[~(type1_2['possible_error_sent'].str.contains('开标') & type1_2['sent'].str.contains('2024'))]
type1_2 = type1_2[~(type1_2['possible_error_sent'].str.contains('标书') & type1_2['sent'].str.contains('2024'))]
type1_2 = type1_2[~(type1_2['possible_error_sent'].str.contains('原料') & type1_2['sent'].str.contains('2024'))]
type1_2 = type1_2[~(type1_2['possible_error_sent'].str.contains('江南') & type1_2['sent'].str.contains('2024'))]
type1_2 = type1_2[~(type1_2['sent'].str.contains('2023年') & ~type1_2['sent'].str.contains('0日'))]
white_list_in = ['投标有效期', '审计', '预算', 'OPEC', '2021年01月01日', '2021年1月1日', '2024年Q1', '2024年第一季度', '2024年一季度', '2024年五一', '同期相比', '同比', '重卡', '物流', '并网', '投标截止时间', '票房', '传媒', '2019年', '2024年工作会议', '质保期', '投标人提前2日']
type1_2 = type1_2[~type1_2['sent'].str.contains('|'.join(white_list_in))]
type1_2 = type1_2.reset_index(drop=True)